In [2]:
import read_tables as rdt 
import plot_atac_signal as pas 

now I write stuff


In [5]:
exp1_dfs_list = rdt.read_experiment(exp_name='exp1')
gfp_0, gfp_1, gfp_2, gfp_4, oma1_0, oma1_1, oma1_2, oma1_4 = exp1_dfs_list

dic_groups = pas.generate_mock_groups()

dic_means = pas.mean_gene_groups_of_sample(sample_df=gfp_0, dic_groups = dic_groups)
pas.plot_groups(dic_means)

NameError: name &#39;gfp_0&#39; is not defined